Before you can store the acquired flights landing data you will have to create the corresponding table in order to obtain the data. I am using MySQLWorkbench and I have created the table the following way:


```
CREATE TABLE flights (
    flight_id INT AUTO_INCREMENT PRIMARY KEY,
    arrival_airport_icao VARCHAR(4),
    departure_airport_icao VARCHAR(4),
    scheduled_arrival_time DATETIME,
    flight_number VARCHAR(30),
    data_retrieved_at DATETIME,
    FOREIGN KEY (arrival_airport_icao) REFERENCES airports(icao_code)
);
```



In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from sqlalchemy import create_engine

def retrieve_and_store_flight_data(icao_list, db_username, db_password, db_host, db_port, db_name):
    # Set API Key and Berlin timezone
    api_key = "YOUR_API_KEY"
    berlin_timezone = timezone('Europe/Berlin')

    # Calculate today and tomorrow dates
    today = datetime.now(berlin_timezone).date()
    tomorrow = today + timedelta(days=1)

    flight_items = []

    # Function to fetch flight data
    def get_flight_data(icao):
        times = [["00:00", "11:59"], ["12:00", "23:59"]]
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false"
            }
            headers = {
                'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
                'x-rapidapi-key': api_key
            }
            response = requests.get(url, headers=headers, params=querystring)
            flights_json = response.json()
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flights_json["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieved_at": retrieval_time
                }
                flight_items.append(flight_item)

    # Fetch data for all ICAOs
    for icao in icao_list:
        get_flight_data(icao)

    # Create DataFrame and process time columns
    flights_df = pd.DataFrame(flight_items)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

    # Create a connection to the database and store the data
    engine = create_engine(f'mysql+mysqlconnector://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')
    flights_df.to_sql('flights', con=engine, if_exists='append', index=False)

    return flights_df

# Example usage
icao_list = ["EDDB", "EDDM", "EDDH"]    #EDDB = Airport Berlin, EDDM = Airport Munich, EDDH = Airport Hamburg
db_username = 'root'
db_password = 'YOUR_PASSWORD'
db_host = '127.0.0.1'
db_port = '3306'
db_name = 'gans'

flights_df = retrieve_and_store_flight_data(icao_list, db_username, db_password, db_host, db_port, db_name)

To maximize the potential of your data repository, it's necessary to interrelate all tables. Therefore, you should create an additional table named 'airports' to connect the 'cities' table with the 'flights' table:


```
CREATE TABLE airports (
    icao_code VARCHAR(4) PRIMARY KEY,
    airport_name VARCHAR(255),
    city_id INT,
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
);
```

